In [ ]:
#mount the google drive first
#https://www.youtube.com/watch?v=LNwODJXcvt4 source to train yolov8 model 
from google.colab import drive
drive.mount('content/drive')

In [ ]:
!pip install -q ultralytics roboflow

In [ ]:
#set the roboflow api key
import os 
ROBOFLOW_API_KEY = "ENTER_YOUR_OWN_API_KEY"
os.environ["ROBOFLOW_API_KEY"] = ROBOFLOW_API_KEY

In [ ]:
from roboflow import Roboflow
rf = Roboflow(api_key=os.environ["ROBOFLOW_API_KEY"])

#dataset user and dataset name workspace
#https://universe.roboflow.com/cvsense/basketball_detection-lbacg/dataset/3 dataset I used with 4546 images alot 

#bigger dataset
proj = rf.workspace("cvsense").project("basketball_detection-lbacg")
dataset = proj.version(3).download("yolov8")

#https://universe.roboflow.com/034-ganesh-kumar-m-v-cs-r2lwe/basketball-lhqoe/browse?queryText=&pageSize=50&startingIndex=0&browseQuery=true smaller datset for first try
#smaller datset
# proj = rf.workspace("034-ganesh-kumar-m-v-cs-r2lwe").project("basketball-lhqoe")
# dataset = proj.version(1).download("yolov8")




In [ ]:
#then after download dataset, update the train test and validation set in the yaml config file 
#then after download dataset, update the train test and validation set in the yaml config file
#copy the dataset into google drive so dont need to use apikey each time 
!mkdir -p /content/drive/MyDrive/shot_tracker_model/datasets
!cp -r {dataset.location} /content/drive/MyDrive/shot_tracker_model/datasets/

#reload each time the data dir uncommment when you have downloaded and saved the datasetonce already
# !cp -r /content/drive/MyDrive/shot_tracker_model/datasets/basketball-1 /content/
# DATA_DIR = "/content/basketball-1"  # then train using DATA_DIR/data.yaml

In [ ]:
#then after download dataset, update the train test and validation set in the yaml config file
#copy the dataset into google drive so dont need to use apikey each time 
# !mkdir -p /content/drive/MyDrive/shot_tracker_model/datasets
!cp -r {dataset.location} /content/drive/MyDrive/shot_tracker_model/datasets/

#reload each time the data dir

#reload bigger dataset
# !cp -r /content/drive/MyDrive/shot_tracker_model/datasets/Basketball_Detection_3/content/
# DATA_DIR = "/content/Basketball_Detection_3"  # then train using DATA_DIR/data.yaml


#reload smaller dataset 
# !cp -r /content/drive/MyDrive/shot_tracker_model/datasets/basketball-1 /content/
# DATA_DIR = "/content/basketball-1"  # then train using DATA_DIR/data.yaml

In [ ]:
#train the yolov9 model with our own dataset with the dataset loaded in 
!yolo task=detect mode=train model=yolov8m.pt data={dataset.location}/data.yaml epochs=20 imgsz=640

In [ ]:
#confusion matrix from training
from IPython.display import Image, display
Image(filename=f'/content/runs/detect/train/confusion_matrix.png', width=600)

In [ ]:
#training results graphs 
Image(filename=f'/content/runs/detect/train/results.png', width=600)

In [ ]:
#choose the best model that we have during training and specify the data location of our validation set and run over the validation set 
!yolo task=detect mode=val model=/content/runs/detect/weights/best.pt data={dataset.location}/data.yaml

In [ ]:
#validation results graphs 
Image(filename=f'/content/runs/detect/val/confusion_matrix.png', width=600)

In [ ]:
#now perform prediction on the trained model on the test set
!yolo task=detect mode=predict model=/content/runs/detect/train/weights/best.pt conf=0.5 source={dataset.location}/test/images

In [ ]:
#confusion matrix from testing set
Image(filename=f'/content/runs/detect/test/confusion_matrix.png', width=600)

In [ ]:
import glob
i =0
for image_path in glob.glob(f'/content/runs/detect/predict/*.jpg'):
    if i == 10:
        break
    display(Image(filename=image_path, height=600))
    print("/n")
    i+=1